In [1]:
from langgraph.graph import StateGraph , START ,END 
from langchain_xai import ChatXAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()
model = ChatXAI(model="grok-4")


In [3]:
class BlogState(TypedDict):
    topic: str
    outline: str
    content: str
    

In [6]:
def create_outline(state : BlogState) -> BlogState:
    # fetch title  
    title = state['title']
    # call llm gen outline 
    prompt = f"Create an outline for a blog post on {title}"
    outline = model.invoke(prompt).content
    state['outline'] = outline
    return state

def create_blog(state : BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    prompt = f"Create a blog post on {title} with the following outline {outline}"
    content = model.invoke(prompt).content
    state['content'] = content
    return state
    

In [7]:
graph = StateGraph(BlogState)

#nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

#edges 
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)

workflow = graph.compile()

In [8]:
initial_state = {'title':'Why is AI important in trading'}
final_state = workflow.invoke(initial_state)
print(final_state['content'])

KeyError: 'title'

In [9]:
print(final_state['outline'])

NameError: name 'final_state' is not defined